#### Imports

In [19]:
# Import necessary packages.
from PIL import Image
import pandas as pd
import os
import urllib.request

#### Create DataFrame for Outputs

In [20]:
# Header for our aspects.
header_names = ["date",
                "N_btl", "N_ntl", "N_atl",
                "NE_btl", "NE_ntl", "NE_atl",
                "E_btl", "E_ntl", "E_atl",
                "SE_btl", "SE_ntl", "SE_atl",
                "S_btl", "S_ntl", "S_atl",
                "SW_btl", "SW_ntl", "SW_atl",
                "W_btl", "W_ntl", "W_atl",
                "NW_btl", "NW_ntl", "NW_atl"]

# Add headers.
df = pd.DataFrame(columns=header_names)

#### Grab Images

In [21]:
path = "octagons/"
base_link = "https://www.sierraavalanchecenter.org"
octo_links = ["octo_links_17.csv", "octo_links_18.csv", "octo_links_19.csv"]
aspects = "aspects.csv"

In [22]:
# Download PNGs.
def dl_png(url, file_path, file_name):
    full_path = file_path + file_name
    urllib.request.urlretrieve(url, full_path)

# Read image source links from all three source files.
for octo_link in octo_links:
    octagons = pd.read_csv(octo_link, usecols=["octagon-src"])

    # Loop through image source links.
    for image_link in octagons["octagon-src"]:
        if isinstance(image_link, str):
            url = base_link + image_link
            date = image_link[37:41] + "-" + image_link[41:43] + "-" + image_link[43:45]
            dl_png(url, path, date + ".png")

#### Image dimensions are [202 X 237]

#### Areas of interest are stated in the report document.
There are, in total, 24 components of the image we wish to analyze. We read the pixel values to determine which aspects are triggered.




In [23]:
sections = {"N_btl" : (100, 30), "N_ntl" : (100, 50), "N_atl" : (100, 75),
            "NE_btl" : (148, 51), "NE_ntl" : (131, 67), "NE_atl" : (114, 82),
            "E_btl" : (160, 100), "E_ntl" : (140, 90), "E_atl" : (121, 98),    
            "SE_btl" : (147, 143), "SE_ntl" : (132, 129), "SE_atl" : (115, 114),
            "S_btl" : (100, 165), "S_ntl" : (100, 142), "S_atl" : (100, 122),
            "SW_btl" : (53, 145), "SW_ntl" : (67, 129), "SW_atl" : (86, 113),
            "W_btl" : (33, 98), "W_ntl" : (55, 97), "W_atl" : (76, 98),
            "NW_btl" : (54, 50), "NW_ntl" : (68, 66), "NW_atl" : (82, 81)}

# RGB value for a greyed out pixel. These greyed areas are "triggered" for avalanche danger.
greyRGB = [(152, 152, 152), (153, 153, 153)]

In [24]:
# Iterate through all .png files in our source directory.
# One-hot encode the octagon sections.
for file in os.listdir(path):
    if file.endswith(".png"):
        file_path = f"{path}/{file}"
        im = Image.open(file_path)
        px = im.load()
        # Add date to dataframe.
        row = [file[0:10]]
        # Check if sections are "greyed out".
        for key in sections:
            coordinate = sections[key]
            if px[coordinate] in greyRGB:
                row.append(1)
            else:
                row.append(0)
        df.loc[len(df)] = row   

#### Sort DataFrame by date

In [25]:
df['date'] = pd.to_datetime(df.date, infer_datetime_format = True)
df.sort_values(by = 'date', ascending = True, inplace = True)

# Turn DataFrame into CSV.
df.to_csv(aspects, index=False)